In [24]:
import pandas as pd 
import numpy as np

data = pd.read_csv('df.csv')
data.columns = ['month_code','service_id','user_id','time_spent','number_days']
user_dict = dict(zip( data.user_id.unique(), list(range(0, len(data.user_id.unique())))))

data.user_id = data.user_id.map(lambda x : user_dict[x])

<br></br><br></br>

__groupby 의 활용__ 

- df.groupby([col])  
해당 컬럼을 기준으로 속성별로 데이터를 grouped 함

In [25]:
data.groupby('service_id')


<br></br><br></br>
__hierarchical indexing__

- 계층적 인덱싱 방법
- groupby에서 2개를 인덱스로 잡았을 떄 생성

In [26]:
h_index = data.groupby(['user_id','service_id'])['time_spent'].sum()

__사용법__

In [27]:
h_index.index.names= ['id','contents']

In [23]:
h_index

id     contents
0      1           3467.0
       4           8829.0
       16           275.0
       19          9811.0
       22           132.0
                    ...  
47573  19             2.0
47574  19           217.0
47575  4            137.0
47576  19            44.0
47577  15           638.0
Name: time_spent, Length: 129346, dtype: float64

In [ ]:
h_index.reset_index().head()

In [ ]:
# 인덱스 레별 변환
h_index = data.groupby(['service_id','user_id','month_code'])['time_spent'].sum()
h_index.swaplevel(i = 0 , j = 1).head()

__계층적 인덱스,컬럼 접근, 제거__

df.index.droplevel()  
df.index.levels[]  
df.index.get_level_values()  


df.columns.droplevel()  
df.columns.get_level_values()  
grouped_df.columns.levels[]

<br></br><br></br><br></br>

__level에 따른 인덱스값 연산__

In [ ]:
h_index.sum(level=1).head()

<br></br><br></br>

__grouped 상태__

- key와 value형태로 묶여 있는 제너레이터 형식의 객체
- 컬럼(속성) 의 값들에 따라서 데이터프레임을 묶는다!!  
        ('속성 값',해당하는 데이터 프레임) 형태의 튜플 
- 묶여진 dataframe은 value역할



In [ ]:
grouped = data.groupby(['service_id','user_id','month_code'])

In [ ]:
list(grouped)[:10]

In [ ]:
grouped.get_group((1,11898,202002))

<br></br><br></br>

__grouped 함수에 다양한 적용__

__agg__ 의 활용

In [ ]:
# agg
grouped = data.groupby(['service_id'])
grouped['time_spent'].agg(lambda x : np.quantile(x,0.25)) # lambda를 이용한 함수사용 가능
grouped['time_spent'].agg([np.sum, np.mean]) # 하나의 컬럼에 여러가지 사용가능

__agg 심화 버전__

In [ ]:
grouped = data.groupby(['service_id'])
grouped_df = grouped.agg({ 'time_spent' :[np.mean, np.max, lambda x : np.quantile(x,0.5)],
            'number_days' : ['min','max']})


In [ ]:
grouped['time_spent'].describe().T

<br></br><br></br>

__group 별 통계량들을 개인 데이터에 적용하고 싶을때!!__  

__transform__

In [ ]:
# transform!!
def max_(x): # 여기서 받는 x는 group으로 묶인 데이터프레임이 들어간다.
    return max(x) # group된 데이터들 끼리 개별 row에다 반환
grouped.transform(max_)

__group 별로 개별 통계량에 따른 정규화 사용법__
- transform을 활용하여 개별 row에다가 적용

In [ ]:
grouped = data.groupby(['service_id'])[['time_spent','number_days']]
def get_minmax(x):
    return (x - min(x))/(max(x)) # numpy 행렬 연산이 기반이라 그냥 더하기 하면 벡터 + 스칼라 그리고 스칼라 곱이된다.
grouped.transform(get_minmax)

<br></br><br></br>

__filter의 활용__

- group된 데이터들을 보면서 일정 기준에 맞는 데이터만 반환한다.
- filter안에서의 함수는 boolean이 반환되도록 해야한다.
- 조건문에 따른 결과 반환이 아닌 True, False 만 반환하여서 loc과 같은 적용을 group별로 실행한다.

In [ ]:
def test_(x):
    if len(x) > 3:
        return True
    return False
data.groupby(['service_id'])[['service_id','time_spent','number_days']].filter(test_).head()

<br></br><br></br>

__pivot table과의 차이점__  

groupby는 그룹으로 묶어 주고 다양한? 연산이 가능한편  
반면 Pivot table의 경우 수치 연산에 특화되어 있다.

<br></br><br></br>

__Crosstab__  

user_item_matrix 만들기에 특화가 되어있다. 

In [ ]:
pd.crosstab( index= data.user_id, columns = data.service_id, values = data.time_spent, aggfunc = 'sum')